# ImageNet Performance
> Performance data for the ImageNet benchmark

- toc: true 
- badges: true
- comments: true
- categories: [l3, performance, ImageNet]
- image: images/imagenet_logo.png

# Theoretical Analysis of ImageNet

### Rooflines

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [2]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
rooflines(pd.read_csv("data/processed_csv/rooflines_hardware_neural_networks.csv"), 'imagenet')

alt.LayerChart(...)

### Performance Prediction

In [3]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load imagenet dataset and plot it
heatmap(pd.read_csv("data/processed_csv/performance_prediction_imagenet.csv"), 'lightgrey','Performance Prediction for Imagenet')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for MNIST

In [4]:
#hide_input
%run scripts/altair_plots.py  #get table with the experiments overview
tableOverviewExperiments(['data/overview_experiments_imagenet.csv'])

In [13]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

csv_path = "./data/cleaned_csv/backup.csv"

In [6]:
#hide
#%writefile scripts/utils.py
def norm_by_group(df, column, group_col):
    """ Normalizes pandas series by group """
    df["norm-"+column] = df.groupby(group_col)[column].apply(lambda x: (x / x.max()))
    return df

def select_color(sel, column):
    """ Easy way to set colors based on selection for altair plots
    """
    return alt.condition(sel, 
                      alt.Color(column),
                      alt.value('lightgray'))

def get_pareto_df(df, groupcol, xcol, ycol):
    pareto_line_df = df.groupby(groupcol)[xcol].max().to_frame("x")
    pareto_line_df['y'] = df.groupby(groupcol)[ycol].agg(lambda x: x.value_counts().index[0])
    pareto_line_df.sort_values('y', ascending=False, inplace=True)
    pareto_line_df['x'] = pareto_line_df.x.cummax()
    pareto_line_df.drop_duplicates('x', keep='first', inplace=True)
    pareto_line_df['group'] = pareto_line_df.index
    return pareto_line_df

def label_point(x, y, val, ax, rot=0):
    """ from https://stackoverflow.com/questions/46027653/adding-labels-in-x-y-scatter-plot-with-seaborn"""
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']), rotation=rot)

In [7]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
imagenet_df = maxp_df[maxp_df.NN_Topology.isin(['GNv1','RN50','MNv1']) & maxp_df["top1 [%]"].notna() & maxp_df['lat-comp'].notna()]
bad_precisions = ["FP"+str(i) for i in range(17,24)]
imagenet_df.Precision = imagenet_df.Precision.apply(lambda x: 'FP16' if x in bad_precisions else x)
imagenet_df["hw_quant_prun_net"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor, r.NN_Topology]), axis=1)
imagenet_df["PruningFactor"] = imagenet_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(imagenet_df, "lat-comp", "NN_Topology");
imagenet_df["quant_model"] = imagenet_df.Precision + '_' + imagenet_df.HWType
imagenet_df.rename(columns={"top1 [%]": "top1"}, inplace=True)
imagenet_df["hw_precision_net_prun"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.NN_Topology, str(r.PruningFactor)]), axis=1)

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

### Line Plot

In [8]:
#hide_input

sel = alt.selection_multi(fields=["hw_quant_prun_net"], bind="legend")
fig25_dot = alt.Chart(imagenet_df).mark_point().encode(
    x='lat-comp',
    y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
    color=select_color(sel, 'hw_quant_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)
fig25_line = alt.Chart(imagenet_df).mark_line().encode(
    x='lat-comp',
    y='fps-comp',
    color=select_color(sel, 'hw_quant_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)

fig = (fig25_dot+fig25_line).properties(
    title="Latency versus Performance for Pruned and Quantized Imagenet Classifier Variants",
    width=W,
    height=H,
).add_selection(sel).interactive()

fig

alt.LayerChart(...)

### Boxplots

In [9]:
#hide_input
box1 = alt.Chart(imagenet_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("lat-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Latency by Hardware/Framework and Pruning for imagenet"
).interactive()
box1

alt.FacetChart(...)

In [10]:
#hide_input
box1 = alt.Chart(imagenet_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("fps-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Throughput by Hardware/Framework and Pruning for imagenet",
).interactive()
box1

alt.FacetChart(...)

## Pareto Graphs

In [11]:
#hide_input
imagenet_pareto = get_pareto_df(imagenet_df, 'hw_precision_net_prun', 'fps-comp', 'top1')
imagenet_lines = alt.Chart(imagenet_df).mark_line(point=True).encode(
    x="fps-comp",
    y=alt.Y("top1:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("hw_precision_net_prun", legend=alt.Legend(columns=1)),
    tooltip=["HWType", "Precision", "PruningFactor", "batch/thread/stream", "top1", "fps-comp"],
)
imagenet_pareto_plot = alt.Chart(imagenet_pareto).mark_line().encode(
    x="x",
    y=alt.Y("y", scale=alt.Scale(zero=False)),
)
(imagenet_lines+imagenet_pareto_plot).interactive().properties(
    width=W,
    height=H,
    title="ImageNet Cassification Design Space: Accuracy versus Performance"
)

alt.LayerChart(...)

# Measurements

Lorem ipsum ...

In [12]:
#hide_input
imagenet_df.describe()

,batch/thread/stream,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],BasePWR [W],IdlePWR [W],FullPwr [W],GOPS,PruningFactor,norm-lat-comp
count,108.000000,108.000000,108.000000,103.000000,108.000000,108.000000,108.000000,108.000000,108.000000,100.000000,108.000000,0.0,108.000000,108.000000
mean,14.509259,268.169789,85.634377,120.124102,429.990518,526.763382,71.327311,90.373770,5.152333,8.970120,13.537268,NaN,84.259259,0.084496
std,28.698398,796.349653,79.038255,122.346505,378.324824,432.906072,2.887991,1.483093,6.532648,8.976935,11.391941,NaN,30.492950,0.192479
min,1.000000,2.570470,16.936600,17.756000,98.644084,112.536576,66.928000,87.705800,0.253000,0.253000,0.462000,NaN,12.500000,0.001783
25%,2.000000,22.021050,37.291575,38.438350,180.599256,185.100699,69.410000,89.260000,1.800000,4.700000,8.239317,NaN,100.000000,0.006964
50%,5.000000,68.489050,59.201100,66.848600,282.710500,405.891000,72.530000,90.850000,2.500000,6.800000,8.361605,NaN,100.000000,0.018346
75%,8.000000,176.938500,116.282250,144.322000,543.816500,819.351390,73.300000,91.400000,2.500000,6.800000,11.040000,NaN,100.000000,0.045650
max,128.000000,7019.100000,379.109000,544.702000,1496.035640,1704.920000,75.172000,92.118000,20.000000,29.000000,40.600000,NaN,100.000000,1.000000
